<a href="https://colab.research.google.com/github/Chaalal-anfel/search-engine/blob/master/model_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing


installation des bibliotèque nécessaire !!!!
 

# Nouvelle section

2. importation et lecture de fichier de données:

In [62]:
df = pd.read_csv('pos_data_en_march.csv')

In [63]:
print(df)


                   temps    alt      lat       lon   sog    cog  eng_stat  \
0    2022-03-19 16:45:02  512.3  35.1892 -0.618785   0.0  108.6         1   
1    2022-03-19 16:45:06  512.3  35.1892 -0.618785   0.0  108.6         1   
2    2022-03-19 16:48:12  482.6  35.1916 -0.617493  23.3   80.4         1   
3    2022-03-19 16:48:37  479.6  35.1917 -0.615918  21.6   83.8         1   
4    2022-03-19 18:17:41  469.6  35.1920 -0.614705   0.0  195.9         1   
..                   ...    ...      ...       ...   ...    ...       ...   
348  2022-03-20 12:09:19  488.2  35.1898 -0.617778   0.0  100.9         1   
349  2022-03-20 12:09:21  488.2  35.1898 -0.617778   0.0  100.9         1   
350  2022-03-20 12:09:26  503.5  35.1895 -0.617638  30.1  159.4         1   
351  2022-03-20 12:09:36  504.0  35.1890 -0.617675  27.8  249.0         1   
352  2022-03-20 12:10:01  488.8  35.1889 -0.618786  22.7  331.7         1   

     error  panic_bt_stat  station_stat  
0        1              0        

3. description des colonnes du **dataframe** :

In [65]:
print('-'*50)
print("Columns in data")
print('-'*50)
print(df.columns.values)

--------------------------------------------------
Columns in data
--------------------------------------------------
['temps' 'alt' 'lat' 'lon' 'sog' 'cog' 'eng_stat' 'error' 'panic_bt_stat'
 'station_stat']


In [66]:
df.head()

,temps,alt,lat,lon,sog,cog,eng_stat,error,panic_bt_stat,station_stat
0,2022-03-19 16:45:02,512.3,35.1892,-0.618785,0.0,108.6,1,1,0,0
1,2022-03-19 16:45:06,512.3,35.1892,-0.618785,0.0,108.6,1,1,0,0
2,2022-03-19 16:48:12,482.6,35.1916,-0.617493,23.3,80.4,1,1,0,0
3,2022-03-19 16:48:37,479.6,35.1917,-0.615918,21.6,83.8,1,1,0,0
4,2022-03-19 18:17:41,469.6,35.1920,-0.614705,0.0,195.9,1,0,0,0


3. faire les modifications nécessaire 
   - changer le type de la colonne **temps** from string to datetime 

In [67]:
print(df['temps'])

0      2022-03-19 16:45:02
1      2022-03-19 16:45:06
2      2022-03-19 16:48:12
3      2022-03-19 16:48:37
4      2022-03-19 18:17:41
              ...         
348    2022-03-20 12:09:19
349    2022-03-20 12:09:21
350    2022-03-20 12:09:26
351    2022-03-20 12:09:36
352    2022-03-20 12:10:01
Name: temps, Length: 353, dtype: object


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   temps          353 non-null    object 
 1   alt            353 non-null    float64
 2   lat            353 non-null    float64
 3   lon            353 non-null    float64
 4   sog            353 non-null    float64
 5   cog            353 non-null    float64
 6   eng_stat       353 non-null    int64  
 7   error          353 non-null    int64  
 8   panic_bt_stat  353 non-null    int64  
 9   station_stat   353 non-null    int64  
dtypes: float64(5), int64(4), object(1)
memory usage: 27.7+ KB


In [69]:
df['temps']= pd.to_datetime(df['temps'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   temps          353 non-null    datetime64[ns]
 1   alt            353 non-null    float64       
 2   lat            353 non-null    float64       
 3   lon            353 non-null    float64       
 4   sog            353 non-null    float64       
 5   cog            353 non-null    float64       
 6   eng_stat       353 non-null    int64         
 7   error          353 non-null    int64         
 8   panic_bt_stat  353 non-null    int64         
 9   station_stat   353 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(4)
memory usage: 27.7 KB


4. calcule de la différence en terme de seconde de chaque deux lignes succéssives de la colonne temps :

In [70]:
df['temps'] = df['temps'].view(int).div(1e9).diff().fillna(0).abs()
# la fonction abs() sera utile si l'ordre de fichier est descendant par rapport au collone temps
df['temps']=df.temps.abs()
print(df)

      temps    alt      lat       lon   sog    cog  eng_stat  error  \
0       0.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
1       4.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
2     186.0  482.6  35.1916 -0.617493  23.3   80.4         1      1   
3      25.0  479.6  35.1917 -0.615918  21.6   83.8         1      1   
4    5344.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
..      ...    ...      ...       ...   ...    ...       ...    ...   
348   163.0  488.2  35.1898 -0.617778   0.0  100.9         1      0   
349     2.0  488.2  35.1898 -0.617778   0.0  100.9         1      0   
350     5.0  503.5  35.1895 -0.617638  30.1  159.4         1      1   
351    10.0  504.0  35.1890 -0.617675  27.8  249.0         1      0   
352    25.0  488.8  35.1889 -0.618786  22.7  331.7         1      0   

     panic_bt_stat  station_stat  
0                0             0  
1                0             0  
2                0             0  
3      

5. calcule de l'acceleration en foction de la variation de la vitesse et le temps:
- calcule de la variation de vitesse entre chaque 2 point en fonction de la vitesse moyenne et la vitesse maximale
- calcule de l'accéleration en terme de deux type de vitesse en ma convertant en seconds


In [71]:
df['sog_dif'] = df['sog'].diff().fillna(0).abs()
print(df)

      temps    alt      lat       lon   sog    cog  eng_stat  error  \
0       0.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
1       4.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
2     186.0  482.6  35.1916 -0.617493  23.3   80.4         1      1   
3      25.0  479.6  35.1917 -0.615918  21.6   83.8         1      1   
4    5344.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
..      ...    ...      ...       ...   ...    ...       ...    ...   
348   163.0  488.2  35.1898 -0.617778   0.0  100.9         1      0   
349     2.0  488.2  35.1898 -0.617778   0.0  100.9         1      0   
350     5.0  503.5  35.1895 -0.617638  30.1  159.4         1      1   
351    10.0  504.0  35.1890 -0.617675  27.8  249.0         1      0   
352    25.0  488.8  35.1889 -0.618786  22.7  331.7         1      0   

     panic_bt_stat  station_stat  sog_dif  
0                0             0      0.0  
1                0             0      0.0  
2              

In [72]:
df['acceleration']=df['sog_dif']*0.27778/df['temps']
print(df)

      temps    alt      lat       lon   sog    cog  eng_stat  error  \
0       0.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
1       4.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
2     186.0  482.6  35.1916 -0.617493  23.3   80.4         1      1   
3      25.0  479.6  35.1917 -0.615918  21.6   83.8         1      1   
4    5344.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
..      ...    ...      ...       ...   ...    ...       ...    ...   
348   163.0  488.2  35.1898 -0.617778   0.0  100.9         1      0   
349     2.0  488.2  35.1898 -0.617778   0.0  100.9         1      0   
350     5.0  503.5  35.1895 -0.617638  30.1  159.4         1      1   
351    10.0  504.0  35.1890 -0.617675  27.8  249.0         1      0   
352    25.0  488.8  35.1889 -0.618786  22.7  331.7         1      0   

     panic_bt_stat  station_stat  sog_dif  acceleration  
0                0             0      0.0           NaN  
1                0             

6. calcule de la distance en fonction des parametre latitude et longitude :
- convertion en radian:

In [73]:
df['LAT_rad'], df['LON_rad'] = np.radians(df['lat']), np.radians(df['lon'])


- calcule de distance:


In [74]:
print(df.head(30))

     temps    alt      lat       lon   sog    cog  eng_stat  error  \
0      0.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
1      4.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
2    186.0  482.6  35.1916 -0.617493  23.3   80.4         1      1   
3     25.0  479.6  35.1917 -0.615918  21.6   83.8         1      1   
4   5344.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
5      3.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
6     49.0  481.1  35.1922 -0.614771  20.5  349.2         1      1   
7     35.0  481.5  35.1930 -0.614560  22.4   79.1         1      0   
8     44.0  475.4  35.1927 -0.612393  21.7  179.8         1      0   
9     36.0  488.4  35.1908 -0.612390   2.5  186.7         1      0   
10    44.0  494.6  35.1888 -0.612826  21.4  210.5         1      0   
11    31.0  475.7  35.1882 -0.613743  22.5  295.1         1      0   
12    25.0  472.6  35.1888 -0.615095  20.1  329.9         1      0   
13    51.0  478.9  3

In [75]:
# vectorized haversine function
import numpy as np
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    #if to_radians:
    #   lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


df['dist'] = \
    haversine(df.LAT_rad.shift(), df.LON_rad.shift(),
                 df.loc[1:, 'LAT_rad'], df.loc[1:, 'LON_rad'])

In [76]:
print(df.head(30))

     temps    alt      lat       lon   sog    cog  eng_stat  error  \
0      0.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
1      4.0  512.3  35.1892 -0.618785   0.0  108.6         1      1   
2    186.0  482.6  35.1916 -0.617493  23.3   80.4         1      1   
3     25.0  479.6  35.1917 -0.615918  21.6   83.8         1      1   
4   5344.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
5      3.0  469.6  35.1920 -0.614705   0.0  195.9         1      0   
6     49.0  481.1  35.1922 -0.614771  20.5  349.2         1      1   
7     35.0  481.5  35.1930 -0.614560  22.4   79.1         1      0   
8     44.0  475.4  35.1927 -0.612393  21.7  179.8         1      0   
9     36.0  488.4  35.1908 -0.612390   2.5  186.7         1      0   
10    44.0  494.6  35.1888 -0.612826  21.4  210.5         1      0   
11    31.0  475.7  35.1882 -0.613743  22.5  295.1         1      0   
12    25.0  472.6  35.1888 -0.615095  20.1  329.9         1      0   
13    51.0  478.9  3

In [78]:
print(df.loc[:,['temps','sog','dist']])

      temps   sog      dist
0       0.0   0.0       NaN
1       4.0   0.0  0.000000
2     186.0  23.3  0.291553
3      25.0  21.6  0.143554
4    5344.0   0.0  0.115164
..      ...   ...       ...
348   163.0   0.0  0.107174
349     2.0   0.0  0.000000
350     5.0  30.1  0.035702
351    10.0  27.8  0.055699
352    25.0  22.7  0.101572

[353 rows x 3 columns]
